In [4]:
import os
import wave
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
from scipy.io.wavfile import read, write
from scipy.signal import fftconvolve

# Constants
INPUT_DIR = 'sound'
OUTPUT_DIR = 'spectral_sound'
ADJUSTMENT_VALUE = 0.5  # Default adjustment value

# Ensure output directory exists
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

def process_wav(file_path, adjustment_value):
    """Apply spectral effect (simulated) to a .wav file."""
    rate, data = read(file_path)
    
    # Simulate spectral effect by applying a convolution with a kernel
    kernel = np.array([1, -adjustment_value, adjustment_value])  # Simple kernel
    processed_data = fftconvolve(data, kernel, mode='same')
    
    # Normalize the processed data
    processed_data = np.int16(processed_data / np.max(np.abs(processed_data)) * 32767)
    
    return rate, processed_data

def process_files():
    files = [f for f in os.listdir(INPUT_DIR) if f.endswith('.wav')]
    
    for idx, file_name in enumerate(files):
        file_path = os.path.join(INPUT_DIR, file_name)
        rate, processed_data = process_wav(file_path, ADJUSTMENT_VALUE)
        
        output_file_name = f"processed_{idx+1}.wav"
        output_file_path = os.path.join(OUTPUT_DIR, output_file_name)
        
        write(output_file_path, rate, processed_data)
        print(f"Processed {file_name} -> {output_file_name}")

def select_files():
    global INPUT_DIR
    files = filedialog.askopenfilenames(filetypes=[("WAV files", "*.wav")])
    if files:
        INPUT_DIR = os.path.dirname(files[0])
        process_files()
        messagebox.showinfo("Processing Complete", "Files have been processed and saved.")

def update_adjustment_value(value):
    global ADJUSTMENT_VALUE
    ADJUSTMENT_VALUE = float(value)
    print(f"Adjustment value set to {ADJUSTMENT_VALUE}")

def create_gui():
    root = tk.Tk()
    root.title("Spectral Sound Processor")
    
    frame = tk.Frame(root, padx=10, pady=10)
    frame.pack(padx=10, pady=10)

    tk.Label(frame, text="Select .wav files to process:").grid(row=0, column=0, pady=(0, 10))
    tk.Button(frame, text="Select Files", command=select_files).grid(row=0, column=1, pady=(0, 10))

    tk.Label(frame, text="Adjustment Value:").grid(row=1, column=0, pady=(10, 0))
    adjustment_slider = tk.Scale(frame, from_=0.0, to=1.0, resolution=0.01, orient=tk.HORIZONTAL, command=update_adjustment_value)
    adjustment_slider.set(ADJUSTMENT_VALUE)
    adjustment_slider.grid(row=1, column=1, pady=(10, 0))
    
    root.mainloop()

if __name__ == "__main__":
    create_gui()


Adjustment value set to 0.5


In [ ]:
import os
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
from scipy.io.wavfile import read, write
from scipy.fft import fft, ifft
from scipy.ndimage import gaussian_filter1d

# Constants
INPUT_DIR = 'sound'
OUTPUT_DIR = 'spectral_sound'
BLEND_FACTOR = 0.5  # Default blend factor

# Ensure output directory exists
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

def spectral_blend(file_path, blend_factor):
  #  """Apply spectral blend effect to a .wav file."""

    rate, data = read(file_path)
    if data.ndim == 2:
        data = data.mean(axis=1)  # Convert to mono if stereo
    
    # Perform FFT
    spectrum = fft(data)
    
    # Apply Gaussian blur to the magnitude of the spectrum
    magnitude = np.abs(spectrum)
    phase = np.angle(spectrum)
    blurred_magnitude = gaussian_filter1d(magnitude, sigma=blend_factor * len(magnitude))
    
    # Reconstruct the blurred spectrum
    blended_spectrum = blurred_magnitude * np.exp(1j * phase)
    
    # Perform inverse FFT
    blended_data = np.real(ifft(blended_spectrum))
    
    # Normalize the processed data
    blended_data = np.int16( blended_data / np.max(np.abs(blended_data)) * 32767)
    

    
    return rate, blended_data




def process_files():
    if not OUTPUT_DIR:
        messagebox.showwarning("Output Directory Not Set", "Please set the output directory first.")
        return
    
    files = [f for f in os.listdir(INPUT_DIR) if f.endswith('.wav')]
    
    for idx, file_name in enumerate(files):
        file_path = os.path.join(INPUT_DIR, file_name)
        rate, processed_data = spectral_blend(file_path, BLEND_FACTOR)
        
        output_file_name = f"processed_{idx+1}.wav"
        output_file_path = os.path.join(OUTPUT_DIR, output_file_name)
        
        write(output_file_path, rate, processed_data)
        print(f"Processed {file_name} -> {output_file_name}")

    messagebox.showinfo("Processing Complete", "Files have been processed and saved.")

def select_files():
    global INPUT_DIR
    files = filedialog.askopenfilenames(filetypes=[("WAV files", "*.wav")])
    if files:
        INPUT_DIR = os.path.dirname(files[0])
        process_files()

def select_output_dir():
    global OUTPUT_DIR
    OUTPUT_DIR = filedialog.askdirectory()
    if OUTPUT_DIR:
        print(f"Output directory set to {OUTPUT_DIR}")

def update_blend_factor(value):
    global BLEND_FACTOR
    BLEND_FACTOR = float(value)
    print(f"Blend factor set to {BLEND_FACTOR}")

def create_gui():
    root = tk.Tk()
    root.title("Spectral Sound Processor")
    
    frame = tk.Frame(root, padx=10, pady=10)
    frame.pack(padx=10, pady=10)

    tk.Label(frame, text="Select .wav files to process:").grid(row=0, column=0, pady=(0, 10))
    tk.Button(frame, text="Select Files", command=select_files).grid(row=0, column=1, pady=(0, 10))

    tk.Label(frame, text="Set Output Directory:").grid(row=1, column=0, pady=(10, 0))
    tk.Button(frame, text="Select Output Directory", command=select_output_dir).grid(row=1, column=1, pady=(10, 0))

    tk.Label(frame, text="Blend Factor:").grid(row=2, column=0, pady=(10, 0))
    blend_slider = tk.Scale(frame, from_=0.0, to=1.0, resolution=0.01, orient=tk.HORIZONTAL, command=update_blend_factor)
    blend_slider.set(BLEND_FACTOR)
    blend_slider.grid(row=2, column=1, pady=(10, 0))
    
    root.mainloop()

if __name__ == "__main__":
    create_gui()


Blend factor set to 0.5
Output directory set to C:/Users/Ming/Downloads/SAve this
Blend factor set to 0.53
Blend factor set to 0.49
Blend factor set to 0.44
Blend factor set to 0.41
Blend factor set to 0.38
Blend factor set to 0.35
Blend factor set to 0.34
Blend factor set to 0.31
Blend factor set to 0.28
Blend factor set to 0.25
Blend factor set to 0.24


In [ ]:
v2

In [ ]:
import os
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
from scipy.io.wavfile import read, write
from scipy.fft import fft, ifft
from scipy.ndimage import gaussian_filter1d

# Constants
INPUT_DIR = 'sound'
OUTPUT_DIR = 'spectral_sound'
BLEND_FACTOR = 0.5  # Default blend factor
FILE_PREFIX = 'processed_'  # Default file prefix

# Ensure output directory exists
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

def spectral_blend(file_path, blend_factor):
    """Apply spectral blur effect to a .wav file."""
    rate, data = read(file_path)
    if data.ndim == 2:
        data = data.mean(axis=1)  # Convert to mono if stereo
    
    # Perform FFT
    spectrum = fft(data)
    
    # Apply Gaussian blur to the magnitude of the spectrum
    magnitude = np.abs(spectrum)
    phase = np.angle(spectrum)
    blurred_magnitude = gaussian_filter1d(magnitude, sigma=blend_factor * len(magnitude))
    
    # Reconstruct the blurred spectrum
    blurred_spectrum = blurred_magnitude * np.exp(1j * phase)
    
    # Perform inverse FFT
    blurred_data = np.real(ifft(blurred_spectrum))
    
    # Normalize the processed data
    blurred_data = np.int16(blurred_data / np.max(np.abs(blurred_data)) * 32767)
    
    return rate, blurred_data

def process_files():
    if not OUTPUT_DIR:
        messagebox.showwarning("Output Directory Not Set", "Please set the output directory first.")
        return
    
    files = [f for f in os.listdir(INPUT_DIR) if f.endswith('.wav')]
    
    for idx, file_name in enumerate(files):
        file_path = os.path.join(INPUT_DIR, file_name)
        rate, processed_data = spectral_blend(file_path, BLEND_FACTOR)
        
        output_file_name = f"{FILE_PREFIX}{idx+1}.wav"
        output_file_path = os.path.join(OUTPUT_DIR, output_file_name)
        
        write(output_file_path, rate, processed_data)
        print(f"Processed {file_name} -> {output_file_name}")

    messagebox.showinfo("Processing Complete", "Files have been processed and saved.")

def select_files():
    global INPUT_DIR
    files = filedialog.askopenfilenames(filetypes=[("WAV files", "*.wav")])
    if files:
        INPUT_DIR = os.path.dirname(files[0])
        process_files()

def select_output_dir():
    global OUTPUT_DIR
    OUTPUT_DIR = filedialog.askdirectory()
    if OUTPUT_DIR:
        print(f"Output directory set to {OUTPUT_DIR}")

def update_blend_factor(value):
    global BLEND_FACTOR
    BLEND_FACTOR = float(value)
    print(f"Blend factor set to {BLEND_FACTOR}")

def update_file_prefix(prefix):
    global FILE_PREFIX
    FILE_PREFIX = prefix
    print(f"File prefix set to {FILE_PREFIX}")

def create_gui():
    root = tk.Tk()
    root.title("Spectral Sound Processor")
    
    frame = tk.Frame(root, padx=10, pady=10)
    frame.pack(padx=10, pady=10)

    tk.Label(frame, text="Select .wav files to process:").grid(row=0, column=0, pady=(0, 10))
    tk.Button(frame, text="Select Files", command=select_files).grid(row=0, column=1, pady=(0, 10))

    tk.Label(frame, text="Set Output Directory:").grid(row=1, column=0, pady=(10, 0))
    tk.Button(frame, text="Select Output Directory", command=select_output_dir).grid(row=1, column=1, pady=(10, 0))

    tk.Label(frame, text="Blend Factor:").grid(row=2, column=0, pady=(10, 0))
    blend_slider = tk.Scale(frame, from_=0.0, to=1.0, resolution=0.01, orient=tk.HORIZONTAL, command=update_blend_factor)
    blend_slider.set(BLEND_FACTOR)
    blend_slider.grid(row=2, column=1, pady=(10, 0))

    tk.Label(frame, text="File Prefix:").grid(row=3, column=0, pady=(10, 0))
    file_prefix_entry = tk.Entry(frame)
    file_prefix_entry.insert(0, FILE_PREFIX)
    file_prefix_entry.grid(row=3, column=1, pady=(10, 0))
    file_prefix_entry.bind("<KeyRelease>", lambda event: update_file_prefix(file_prefix_entry.get()))
    
    root.mainloop()

if __name__ == "__main__":
    create_gui()


Blend factor set to 0.5
Blend factor set to 0.47
Blend factor set to 0.43
Blend factor set to 0.4
Blend factor set to 0.37
Blend factor set to 0.35
Blend factor set to 0.32
Output directory set to C:/Users/Ming/Downloads/SAve this


d902a451-dde1-41d5-8863-4abfdbcba adding multi threading

In [7]:
import os
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
from threading import Thread
from scipy.io.wavfile import read, write
from scipy.fft import fft, ifft
from scipy.ndimage import gaussian_filter1d

# Constants
INPUT_DIR = 'sound'
OUTPUT_DIR = 'spectral_sound'
BLEND_FACTOR = 0.5  # Default blend factor
FILE_PREFIX = 'processed_'  # Default file prefix

# Ensure output directory exists
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

def spectral_blend(file_path, blend_factor):
    """Apply spectral blur effect to a .wav file."""
    rate, data = read(file_path)
    if data.ndim == 2:
        data = data.mean(axis=1)  # Convert to mono if stereo
    
    # Perform FFT
    spectrum = fft(data)
    
    # Apply Gaussian blur to the magnitude of the spectrum
    magnitude = np.abs(spectrum)
    phase = np.angle(spectrum)
    blurred_magnitude = gaussian_filter1d(magnitude, sigma = blend_factor * len(magnitude))
    
    # Reconstruct the blurred spectrum
    blurred_spectrum = blurred_magnitude * np.exp(1j * phase)
    
    # Perform inverse FFT
    blurred_data = np.real(ifft(blurred_spectrum))
    
    # Normalize the processed data
    blurred_data = np.int16(blurred_data / np.max(np.abs(blurred_data)) * 32767)
    
    return rate, blurred_data

def process_files():
    if not OUTPUT_DIR:
        messagebox.showwarning("Output Directory Not Set", "Please set the output directory first.")
        return
    
    files = [f for f in os.listdir(INPUT_DIR) if f.endswith('.wav')]
    
    for idx, file_name in enumerate(files):
        file_path = os.path.join(INPUT_DIR, file_name)
        rate, processed_data = spectral_blend(file_path, BLEND_FACTOR)
        
        output_file_name = f"{FILE_PREFIX}{idx+1}.wav"
        output_file_path = os.path.join(OUTPUT_DIR, output_file_name)
        
        write(output_file_path, rate, processed_data)
        print(f"Processed {file_name} -> {output_file_name}")

    messagebox.showinfo("Processing Complete", "Files have been processed and saved.")

def select_files():
    global INPUT_DIR
    files = filedialog.askopenfilenames(filetypes=[("WAV files", "*.wav")])
    if files:
        INPUT_DIR = os.path.dirname(files[0])
        process_files_thread = Thread(target=process_files)
        process_files_thread.start()

def select_output_dir():
    global OUTPUT_DIR
    OUTPUT_DIR = filedialog.askdirectory()
    if OUTPUT_DIR:
        print(f"Output directory set to {OUTPUT_DIR}")


def start_processing():
    """Function to start the processing of files in a separate thread."""
    process_files_thread = Thread(target=process_files)
    process_files_thread.start()
    

def go_button(frame):
    """Function to add a 'GO' button to the GUI."""
    tk.Button(frame, text="GO", command=start_processing).grid(row=4, columnspan=2, pady=(10, 0))

def update_blend_factor(value):
    global BLEND_FACTOR
    BLEND_FACTOR = float(value)
    print(f"Blend factor set to {BLEND_FACTOR}")

def update_file_prefix(prefix):
    global FILE_PREFIX
    FILE_PREFIX = prefix
    print(f"File prefix set to {FILE_PREFIX}")


def create_gui():
    root = tk.Tk()
    root.title("Spectral Sound Processor")
    
    frame = tk.Frame(root, padx=10, pady=10)
    frame.pack(padx=10, pady=10)

    tk.Label(frame, text="Select .wav files to process:").grid(row=0, column=0, pady=(0, 10))
    tk.Button(frame, text="Select Files", command=select_files).grid(row=0, column=1, pady=(0, 10))

    tk.Label(frame, text="Set Output Directory:").grid(row=1, column=0, pady=(10, 0))
    tk.Button(frame, text="Select Output Directory", command=select_output_dir).grid(row=1, column=1, pady=(10, 0))

   
    tk.Button(frame, text="Go", command = go_button(frame))
       

    tk.Label(frame, text="Blend Factor:").grid(row=2, column=0, pady=(10, 0))
    blend_slider = tk.Scale(frame, from_=0.0, to=1.0, resolution=0.01, orient=tk.HORIZONTAL, command=update_blend_factor)
    blend_slider.set(BLEND_FACTOR)
    blend_slider.grid(row=2, column=1, pady=(10, 0))

    tk.Label(frame, text="File Prefix:").grid(row=3, column=0, pady=(10, 0))
    file_prefix_entry = tk.Entry(frame)
    file_prefix_entry.insert(0, FILE_PREFIX)
    file_prefix_entry.grid(row=3, column=1, pady=(10, 0))
    file_prefix_entry.bind("<KeyRelease>", lambda event: update_file_prefix(file_prefix_entry.get()))
    
    root.mainloop()



if __name__ == "__main__":
    create_gui()


Blend factor set to 0.5
File prefix set to b
File prefix set to bl
File prefix set to blu
File prefix set to blur
File prefix set to blur2
File prefix set to blur22
Output directory set to C:/Users/Ming/Documents/GitHub/SpectralBitCrush/spectral_sound
Blend factor set to 0.53
Blend factor set to 0.54
Blend factor set to 0.57
Blend factor set to 0.6
Blend factor set to 0.63
Blend factor set to 0.65
Blend factor set to 0.68
Blend factor set to 0.71
Blend factor set to 0.74
Blend factor set to 0.75
Blend factor set to 0.78
Blend factor set to 0.81
Blend factor set to 0.84
Blend factor set to 0.85
Blend factor set to 0.88
Blend factor set to 0.91
Blend factor set to 0.94
Blend factor set to 0.96
Blend factor set to 0.99
Blend factor set to 1.0
Blend factor set to 0.99
Blend factor set to 0.96
Blend factor set to 0.94
Blend factor set to 0.91


Exception in thread Thread-9 (process_files):
Traceback (most recent call last):
  File "C:\Users\Ming\anaconda3\envs\Spectral\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\Users\Ming\anaconda3\envs\Spectral\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Ming\AppData\Local\Temp\ipykernel_2968\1421680896.py", line 50, in process_files
FileNotFoundError: [WinError 3] The system cannot find the path specified: 'sound'


In [ ]:
v5

In [ ]:
import os
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
from threading import Thread
from scipy.io.wavfile import read, write
from scipy.fft import fft, ifft
from scipy.ndimage import gaussian_filter1d

# Constants
INPUT_DIR = 'sound'
OUTPUT_DIR = 'spectral_sound'
BLEND_FACTOR = 0.5  # Default blend factor
FILE_PREFIX = 'processed_'  # Default file prefix

# Ensure output directory exists
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

def spectral_blend(file_path, blend_factor):
    """Apply spectral blur effect to a .wav file."""
    rate, data = read(file_path)
    if data.ndim == 2:
        data = data.mean(axis=1)  # Convert to mono if stereo
    
    # Perform FFT
    spectrum = fft(data)
    
    # Apply Gaussian blur to the magnitude of the spectrum
    magnitude = np.abs(spectrum)
    phase = np.angle(spectrum)
    blurred_magnitude = gaussian_filter1d(magnitude, sigma = blend_factor * len(magnitude))
    
    # Reconstruct the blurred spectrum
    blurred_spectrum = blurred_magnitude * np.exp(1j * phase)
    
    # Perform inverse FFT
    blurred_data = np.real(ifft(blurred_spectrum))
    
    # Normalize the processed data
    blurred_data = np.int16(blurred_data / np.max(np.abs(blurred_data)) * 32767)
    
    return rate, blurred_data

def process_files():
    if not os.path.exists(INPUT_DIR):
        messagebox.showwarning("Input Directory Not Found", f"The directory '{INPUT_DIR}' does not exist.")
        return
    
    if not OUTPUT_DIR:
        messagebox.showwarning("Output Directory Not Set", "Please set the output directory first.")
        return
    
    files = [f for f in os.listdir(INPUT_DIR) if f.endswith('.wav')]
    
    for idx, file_name in enumerate(files):
        file_path = os.path.join(INPUT_DIR, file_name)
        rate, processed_data = spectral_blend(file_path, BLEND_FACTOR)
        
        output_file_name = f"{FILE_PREFIX}{idx+1}.wav"
        output_file_path = os.path.join(OUTPUT_DIR, output_file_name)
        
        write(output_file_path, rate, processed_data)
        print(f"Processed {file_name} -> {output_file_name}")

    messagebox.showinfo("Processing Complete", "Files have been processed and saved.")

def select_files():
    global INPUT_DIR
    files = filedialog.askopenfilenames(filetypes=[("WAV files", "*.wav")])
    if files:
        INPUT_DIR = os.path.dirname(files[0])
        print(f"Input directory set to {INPUT_DIR}")
        process_files_thread = Thread(target=process_files)
        process_files_thread.start()

def select_output_dir():
    global OUTPUT_DIR
    OUTPUT_DIR = filedialog.askdirectory()
    if OUTPUT_DIR:
        print(f"Output directory set to {OUTPUT_DIR}")

def start_processing():
    """Function to start the processing of files in a separate thread."""
    process_files_thread = Thread(target=process_files)
    process_files_thread.start()

def go_button(frame):
    """Function to add a 'GO' button to the GUI."""
    tk.Button(frame, text="GO", command=start_processing).grid(row=4, columnspan=2, pady=(10, 0))

def update_blend_factor(value):
    global BLEND_FACTOR
    BLEND_FACTOR = float(value)
    print(f"Blend factor set to {BLEND_FACTOR}")

def update_file_prefix(prefix):
    global FILE_PREFIX
    FILE_PREFIX = prefix
    print(f"File prefix set to {FILE_PREFIX}")

def create_gui():
    root = tk.Tk()
    root.title("Spectral Sound Processor")
    
    frame = tk.Frame(root, padx=10, pady=10)
    frame.pack(padx=10, pady=10)

    tk.Label(frame, text="Select .wav files to process:").grid(row=0, column=0, pady=(0, 10))
    tk.Button(frame, text="Select Files", command=select_files).grid(row=0, column=1, pady=(0, 10))

    tk.Label(frame, text="Set Output Directory:").grid(row=1, column=0, pady=(10, 0))
    tk.Button(frame, text="Select Output Directory", command=select_output_dir).grid(row=1, column=1, pady=(10, 0))

    tk.Label(frame, text="Blend Factor:").grid(row=2, column=0, pady=(10, 0))
    blend_slider = tk.Scale(frame, from_=0.0, to=1.0, resolution=0.01, orient=tk.HORIZONTAL, command=update_blend_factor)
    blend_slider.set(BLEND_FACTOR)
    blend_slider.grid(row=2, column=1, pady=(10, 0))

    tk.Label(frame, text="File Prefix:").grid(row=3, column=0, pady=(10, 0))
    file_prefix_entry = tk.Entry(frame)
    file_prefix_entry.insert(0, FILE_PREFIX)
    file_prefix_entry.grid(row=3, column=1, pady=(10, 0))
    file_prefix_entry.bind("<KeyRelease>", lambda event: update_file_prefix(file_prefix_entry.get()))

    go_button(frame)
    
    root.mainloop()

if __name__ == "__main__":
    create_gui()


Blend factor set to 0.5
Blend factor set to 0.53
Blend factor set to 0.56
Blend factor set to 0.57
Blend factor set to 0.6
Blend factor set to 0.63
Blend factor set to 0.66
Blend factor set to 0.68
Blend factor set to 0.71
Blend factor set to 0.74
Blend factor set to 0.76
Blend factor set to 0.78
Blend factor set to 0.81
Blend factor set to 0.76
Blend factor set to 0.74
File prefix set to B
File prefix set to B
File prefix set to Bl
File prefix set to Blu
File prefix set to Blur
File prefix set to Blurr
File prefix set to Blurre
File prefix set to Blurred
File prefix set to Blurredd
File prefix set to Blurreddd
Output directory set to C:/Users/Ming/Documents/GitHub/SpectralBitCrush/spectral_sound
Input directory set to C:/Users/Ming/Downloads
Output directory set to C:/Users/Ming/Documents/GitHub/SpectralBitCrush/save
Input directory set to C:/Users/Ming/Downloads
File prefix set to Blurreddd
File prefix set to Blurreddd
File prefix set to Blurreddd
File prefix set to Blurreddd
File pr